In [1]:
#　mass and size distribution function
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from IPython.display import clear_output
from matplotlib.backends.backend_pdf import PdfPages

collection_area = pd.read_csv("collection_area.csv")
df_mu = pd.read_csv("../MU_data_2009-2015.csv")
array_mu = df_mu.values
total_observation_time_hour = 845.8 #[hour]

def massindex(x, a, b):
    return 10**(b-(a-1)*np.log10(x))

def sizeindex(x, a, b):
    return 10**(b-a*np.log10(x))

#RCSが空, RSC>20.0, zenith distance>90.0 のデータを削除
#RCS = -4.58*MAG+16.92 ⇨ MAG = RCS/(-4.58) + 16.92/4.58
rcs = df_mu["RCS"].values
zenith_distance = df_mu["Ze_(deg)"].values
mass = df_mu["mass_grams_calculated"].values
size = df_mu["size_calculated"].values
rcs_unicode = rcs.astype(unicode)
nan = np.array(["nan"])
nan = nan.reshape(1)
nan = nan.astype(unicode)
delete_index = np.where((rcs_unicode == nan) | (rcs > 20.0) | (zenith_distance > 90.0))
array_mu = np.delete(array_mu, delete_index, 0)
mass = np.delete(mass, delete_index, 0)
size = np.delete(size, delete_index, 0)
zenith_distance = np.delete(zenith_distance, delete_index, 0)

################################################################

zenith_correction = 1./np.cos(np.deg2rad(zenith_distance))
mass_bins = np.logspace(np.log10(np.min(mass)),np.log10(np.max(mass)),num=200, dtype=np.float)
size_bins = np.cbrt(6.*mass_bins/(2.*np.pi))/100.
collection_area_table = collection_area["collection_area"].values
mass_axis = np.ones(len(mass_bins)-1)
size_axis = np.ones(len(mass_bins)-1)
number = np.ones(len(mass_bins) -1)
collection_area = np.ones(len(mass_bins) -1)
cumulative_flux = np.ones(len(mass_bins) -1)

for i in range(len(mass_bins)-1):
    index = np.array(np.where((mass > mass_bins[i]) & (mass < mass_bins[i+1])))
    num = zenith_correction[index]
    collection_area[i] = (collection_area_table[i] + collection_area_table[i+1])/2.
    mass_axis[i] = (mass_bins[i] + mass_bins[i+1])/2.
    number[i] = np.sum(num)
    cumulative_flux[i] = number[i]/collection_area[i]/total_observation_time_hour
    size_axis[i] = (size_bins[i] + size_bins[i+1])/2.

number = number[::-1]
cumulative_number = np.cumsum(number)
cumulative_flux_err = np.sqrt(cumulative_number)/collection_area/total_observation_time_hour
cumulative_flux = cumulative_flux[::-1]
cumulative_flux = np.cumsum(cumulative_flux)    
mass_axis = mass_axis[::-1]
size_axis = size_axis[::-1]

################################################################

param_mass, cov_mass = curve_fit(massindex, mass_axis[22:110], cumulative_flux[22:110],\
                                 sigma=cumulative_flux_err[22:110])
y_fit_mass = 10**(param_mass[1]-(param_mass[0]-1)*np.log10(mass_axis))
param_size, cov_size = curve_fit(sizeindex, size_axis[22:110], cumulative_flux[22:110],\
                                 sigma=cumulative_flux_err[22:110])
y_fit_size = 10**(param_size[1] - param_size[0]*np.log10(size_axis))

cumulative_flux_max = cumulative_flux + cumulative_flux_err
cumulative_flux_min = cumulative_flux - cumulative_flux_err

fig = plt.figure(figsize=(10,10))
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["font.size"] = 25
ax1 = fig.add_subplot(111)
ax1.plot(mass_axis,cumulative_flux,color="darkblue",linewidth=4)
ax1.fill_between(mass_axis,cumulative_flux_min,cumulative_flux_max,alpha=0.2)
ax1.plot(mass_axis,y_fit_mass,linewidth=2,linestyle="dashdot",color="darkblue")
ax1.grid(which='major',alpha=0.2,color='black',linestyle='-')
ax1.set_xscale("log")
ax1.set_xlabel("Log Particle Mass[g]")
ax1.set_ylabel("Log Cumulative Flux [$km^{-2} hour^{-1}$]")

ax2 = ax1.twiny()
ax2.plot(size_axis,cumulative_flux,alpha=0.01)
ax2.set_xlabel("Log Particle Diameter[m]")

plt.yscale("log")
plt.xscale("log")
plt.text(10**-3.8,1e-4,"Mass Index s = 1.96$\pm$ 0.00693",color='blue',size=22)
plt.text(10**-3.8,3e-5,"Fitting range [-3:0]",color='blue',size=22)
plt.text(10**-3.2,0.4e1,"Size Index b = 2.87$\pm$ 0.0208",color='red',size=22)
plt.text(10**-2.8,1e0,"Fitting range [-3:-2]",color='red',size=22)
#pdf = PdfPages("d_img/mass_and_size_index.pdf")
#pdf.savefig()
#pdf.close()
plt.savefig("d_img/mass_and_size_index.png")
plt.show()

clear_output()
print "mass index", param_mass, "error", np.sqrt(np.diag(cov_mass))
print "fitting range is", mass_axis[22], ":", mass_axis[110], "10^-3"
print "size index", param_size, "error", np.sqrt(np.diag(cov_size))
print "fitting range is", size_axis[22], ":", size_axis[110]
print "Complete!"

mass index [ 1.95549635 -3.3663456 ] error [0.00693104 0.01969549]
fitting range is 1.0008695321411292 : 0.0009556217848005595 10^-3
size index [ 2.86648897 -9.11867662] error [0.02079309 0.06139028]
fitting range is 0.009848595755472185 : 0.0009697888115479032
Complete!


In [6]:
#　Mass distribution function
df_radio = pd.read_csv("d_data/radio.csv", header=None)
df_visual = pd.read_csv("d_data/visual.csv", header=None)
df_satelite = pd.read_csv("d_data/satelite.csv", header=None)
stradian = 6.2832

array_mu = df_mu.values
array_radio = df_radio.values
array_radio = 10**(array_radio)
array_visual = df_visual.values
array_visual = 10**(array_visual)
array_satelite = df_satelite.values
array_satelite = 10**(array_satelite)
array_radio_x = array_radio[:,0]
array_radio_y = array_radio[:,1]
array_visual_x = array_visual[:,0]
array_visual_y = array_visual[:,1]
array_satelite_x = array_satelite[:,0]
array_satelite_y = array_satelite[:,1]

################################################################

for i in range(len(mass_bins)-1):
    index = np.array(np.where((mass > mass_bins[i]) & (mass < mass_bins[i+1])))
    num = zenith_correction[index]
    collection_area[i] = (collection_area_table[i] + collection_area_table[i+1])/2.
    mass_axis[i] = (mass_bins[i] + mass_bins[i+1])/2.
    number[i] = np.sum(num)
    cumulative_flux[i] = number[i]/(collection_area[i]*10**6)/(total_observation_time_hour*60*60)/stradian

number = number[::-1]
cumulative_number = np.cumsum(number)
cumulative_flux_err = np.sqrt(cumulative_number)/(collection_area*10**6)/(total_observation_time_hour*60*60)/stradian
cumulative_flux = cumulative_flux[::-1]
cumulative_flux = np.cumsum(cumulative_flux)    
mass_axis = mass_axis[::-1]

################################################################

param_mass, cov_mass = curve_fit(massindex, mass_axis[22:110], cumulative_flux[22:110],\
                                 sigma=cumulative_flux_err[22:110])
y_fit_mass = param_mass[1]*mass_axis**(-1*(param_mass[0]-1))
cumulative_flux_max = cumulative_flux + cumulative_flux_err
cumulative_flux_min = cumulative_flux - cumulative_flux_err

fig = plt.figure(figsize=(16,10))
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["font.size"] = 25
plt.plot(mass_axis,cumulative_flux,color="darkblue",linewidth=4)
plt.fill_between(mass_axis,cumulative_flux_min,cumulative_flux_max,alpha=0.2)
plt.plot(mass_axis,y_fit_mass,linewidth=2,linestyle="dashdot",color="darkblue")
plt.grid(which='major',alpha=0.2,color='black',linestyle='-')

plt.fill(array_radio_x,array_radio_y,color="y",alpha=0.5,hatch = '/')
plt.fill(array_visual_x,array_visual_y,color="red",alpha=0.5,hatch = '/')
plt.fill(array_satelite_x,array_satelite_y,color="green",alpha=0.5,hatch = '/')

plt.text(10**-12.5,10**-7,"Satellite, $s = 1.55 \pm 0.02$",size=24)
plt.text(10**-8,10**-13,"Radio, $s = 2.03 \pm 0.02$",size=24)
plt.text(10**-6,10**-8,"This research, s = 1.96$\pm$ 6.93*$10^{-3}$",color='blue',size=24)
plt.text(10**-4.5,10**-16.5,"Visual, $s = 2.62 - 2.89$",size=24)

plt.yscale("log")
plt.xscale("log")
plt.xlabel("Log Particle Mass[g]")
plt.ylabel("Log Cumulative Flux [$m^{-2} sec^{-1} 2\pi str^{-1}$]")
#pdf = PdfPages("d_img/mass_distribution.pdf")
#pdf.savefig()
#pdf.close()
plt.savefig("d_img/mass_distribution.png")
plt.show()

clear_output()
print "Complete!"

Complete!


In [2]:
# Size distribution function
df_fireball = pd.read_csv("d_data/fireball.csv", header=None)
df_gruen = pd.read_csv("d_data/Gruen.csv", header=None)
array_fireball = df_fireball.values
array_gruen = df_gruen.values
array_gruen = 10**(array_gruen)
array_fireball_x = array_fireball[:,0]
array_fireball_y = 10**array_fireball[:,1]
array_gruen_x = array_gruen[:,0]
array_gruen_y = array_gruen[:,1]

radius_of_the_earth = 6378.1 #[km]
surface_area_of_the_earth = 4.*np.pi*radius_of_the_earth**2 #[km^2]
year_hr = 365.*24.

################################################################

for i in range(len(mass_bins)-1):
    index = np.array(np.where((mass > mass_bins[i]) & (mass < mass_bins[i+1])))
    num = zenith_correction[index]
    collection_area[i] = (collection_area_table[i] + collection_area_table[i+1])/2.
    number[i] = np.sum(num)
    cumulative_flux[i] = number[i]/collection_area[i]/total_observation_time_hour\
                        *surface_area_of_the_earth*year_hr
    size_axis[i] = (size_bins[i] + size_bins[i+1])/2.

number = number[::-1]
cumulative_number = np.cumsum(number)
cumulative_flux_err = np.sqrt(cumulative_number)/collection_area/total_observation_time_hour\
                        *surface_area_of_the_earth*year_hr
cumulative_flux = cumulative_flux[::-1]
cumulative_flux = np.cumsum(cumulative_flux)    
size_axis = size_axis[::-1]

################################################################

param_size, cov_size = curve_fit(sizeindex, size_axis[22:110], cumulative_flux[22:110],\
                                 sigma=cumulative_flux_err[22:110])
y_fit_size = 10**(param_size[1] - param_size[0]*np.log10(size_axis))
brown_x = np.logspace(-1.5,9,10)
brown_y = 10**(1.568-2.7*np.log10(brown_x))
cumulative_flux_max = cumulative_flux + cumulative_flux_err
cumulative_flux_min = cumulative_flux - cumulative_flux_err

fig = plt.figure(figsize=(10,10))
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["font.size"] = 25
plt.plot(size_axis,cumulative_flux,color="darkblue",linewidth=2,label="This research")
#plt.fill_between(size_axis,cumulative_flux_min,cumulative_flux_max,alpha=0.2)
plt.plot(size_axis,y_fit_size,linewidth=2,linestyle="dashdot",color="darkblue")
plt.plot(brown_x,brown_y,linewidth=4,linestyle="dashdot",color="green",label="P.Brown(2002)")
plt.plot(array_fireball_x,array_fireball_y,color="orange",linestyle="solid",linewidth=3,label="Halliday(1996)")
plt.plot(array_gruen_x,array_gruen_y,color="purple",linestyle="dashdot",label="Grun(1985)")

plt.yscale("log")
plt.xscale("log")
plt.xlim(10e-6,10e1)
plt.ylim(10e-3,10e18)
plt.legend(loc='upoer right')
plt.grid(which='major',alpha=0.2,color='black',linestyle='-')
plt.xlabel("Log Particle Diameter[m]")
plt.ylabel("Log Cumulative number colliding with the Earth per year")
#pdf = PdfPages("d_img/size_distribution.pdf")
#pdf.savefig()
#pdf.close()
plt.savefig("d_img/size_distribution.png")
plt.show()

clear_output()
print "Complete!"

Complete!
